In [1]:
import numpy as np
import cvxopt
import cvxopt.solvers
from sklearn.metrics.pairwise import linear_kernel
from sklearn.metrics import accuracy_score
from sklearn.datasets import load_svmlight_file

In [2]:
'''
Train linear SVM
Input: 
        X, matrix of size #sample x #feature
        y, vector of size #sample, either -1 or 1
        C, regularizer parameter (default None for hard margin case)
Output:
        w, weight vector of size #feature
        b, intercept, a float
'''
def train_svm(X, y, C=None):
    n_samples, n_features = X.shape

    # Compute Gram matrix
    K = linear_kernel(X)

    # Solve dual quadratic programming problem
    # min{0.5*a.T*P*a+q.T*a}
    # subject to G*a<=h and A*a=b
    P = cvxopt.matrix(np.outer(y,y) * K)
    q = cvxopt.matrix(np.ones(n_samples) * -1)
    A = cvxopt.matrix(y, (1,n_samples))
    b = cvxopt.matrix(0.0)
    if C is None:
        # Hard margin with constraint
        # ai >= 0
        G = cvxopt.matrix(np.diag(np.ones(n_samples) * -1))
        h = cvxopt.matrix(np.zeros(n_samples))
    else:
        # Soft margin with constraint
        # 0 <= ai <= C
        tmp1 = np.diag(np.ones(n_samples) * -1)
        tmp2 = np.identity(n_samples)
        G = cvxopt.matrix(np.vstack((tmp1, tmp2)))
        tmp1 = np.zeros(n_samples)
        tmp2 = np.ones(n_samples) * C
        h = cvxopt.matrix(np.hstack((tmp1, tmp2)))

    # Solve the quadratic programming problem
    cvxopt.solvers.options['show_progress'] = False
    solution = cvxopt.solvers.qp(P, q, G, h, A, b)
    a = np.ravel(solution['x'])

    # Support vectors have non zero Lagrange multipliers
    sv = a > 1e-5
    ind = np.arange(len(a))[sv]
    a = a[sv]
    sv_y = y[sv]
    sv_X = X[sv]

    # Calculate intercept b
    b = 0
    for n in range(len(a)):
        b += sv_y[n]
        b -= np.sum(a * sv_y * K[ind[n],sv])
        b /= len(a)

    # Calculate weight vector w
    w = np.zeros(n_features)
    for n in range(len(a)):
        w += a[n] * sv_y[n] * sv_X[n]

    return w, b


'''
Train a variant of linear SVM
Input: 
        X1, matrix of size #sample1 x #feature
        y1, vector of size #sample1, either -1 or 1
        X2, matrix of size #sample2 x #feature
        y2, vector of size #sample2, either -1 or 1
        C1, regularizer parameter, control the slack variable of samples from X1
        C2, regularizer parameter, control the slack variable of postive samples from X2
        C3, regularizer parameter, control the slack variable of negative samples from X2
Output:
        w, weight vector of size #feature
        b, intercept, a float
        slack1, slack variable of samples from X1, vector of size #sample1
        slack2, slack variable of samples from X2, vector of size #sample2
'''
def train_variantsvm(X1, y1, X2, y2, C1, C2, C3):
	# Concatenate two data set
    X = np.vstack([X1, X2])
    y = np.append(y1, y2)

    # Create indicator vector mark of size #sample, where
    # mark[i] = 0, sample i is from X1
    # mark[i] = 1, sample i is from X2 and yi = 1
    # mark[i] =-1, sample i is from X2 and yi = -1
    n_samples, n_features = X.shape
    mark = y.copy()
    mark[:X1.shape[0]] = 0

    # Compute Gram matrix
    K = linear_kernel(X)

    # Solve dual quadratic programming problem
    # min{0.5*a.T*P*a+q.T*a}
    # subject to G*a<=h and A*a=b
    P = cvxopt.matrix(np.outer(y,y) * K)
    q = cvxopt.matrix(np.ones(n_samples) * -1)
    A = cvxopt.matrix(y, (1,n_samples))
    b = cvxopt.matrix(0.0)

    # Calculate G and h with constraint 
    # for xi in X1, 0 <= ai <= C1
    # for xi in X2 and yi is 1,  0 <= ai <= C2
    # for xi in X2 and yi is -1, 0 <= ai <= C3
    tmp=[]
    for i in [0,1,-1]:
        a=np.zeros(len(mark))
        idx= np.where(mark==i)
        if idx[0].size!=0:
            a[idx]=1
            tmp.append(a)
    C = [C1, C2, C3]
    gi=[]
    hi=[]
    for bi, c in zip(tmp, C):
        tmp1 = np.diag(bi) * -1
        tmp2 = np.diag(bi)
        gi.append(np.vstack((tmp1, tmp2)))
        tmp1 = np.zeros(len(bi))
        tmp2 = np.ones(len(bi)) * c
        hi.append(np.hstack((tmp1, tmp2)))
    G= cvxopt.matrix(np.vstack(gi))
    h= cvxopt.matrix(np.hstack(hi))


    # Solve the quadratic programming problem
    cvxopt.solvers.options['show_progress'] = False
    solution = cvxopt.solvers.qp(P, q, G, h, A, b)
    a = np.ravel(solution['x'])

    # Support vectors have non zero Lagrange multipliers
    sv = a > 1e-5
    ind = np.arange(len(a))[sv]
    a = a[sv]
    sv_y = y[sv]
    sv_X = X[sv]

    # Caculate intercept b
    b = 0
    for n in range(len(a)):
        b += sv_y[n]
        b -= np.sum(a * sv_y * K[ind[n],sv])
        b /= len(a)

    # Calculate weight vector w
    w = np.zeros(n_features)
    for n in range(len(a)):
        w += a[n] * sv_y[n] * sv_X[n]

    # Get slack variable
    slack = np.zeros(n_samples)
    for n in range(len(a)):
        if abs(C1 - a[n]) < 1e-5 or abs(C2 - a[n]) < 1e-5 or abs(C3 - a[n]) < 1e-5:
            slack[ind[n]] = abs(sv_y[n] * (np.dot(sv_X[n], w) + b))    

    return w, b, slack[:X1.shape[0]], slack[X1.shape[0]:]


'''
Train linear transductive SVM
Input: 
        X1, matrix of size #sample1 x #feature
        y1, vector of size #sample1, either -1 or 1
        X2, matrix of size #sample2 x #feature
        C1, regularizer parameter, control the slack variable of samples from X1
        C2, regularizer parameter, control the slack variable of samples from X2
        p,  percentage of positive samples in unlabeled data X2, [0, 1]
Output:
        w, weight vector of size #feature
        b, intercept, a float
'''
def train_tsvm(X1, y1, X2, C1, C2, p):
    
    # 1. Get number of positive samples
    num_pos = int(X2.shape[0] * p)

    # 2. Train standard SVM using labeled samples
    w, b = train_svm(X1, y1, C1)
    
    # 3. The num_pos test examples from X2 with highest value of w*x+b are assigned to 1
    # The rest of examples from X2 are assigned to -1
    val=np.zeros(train_unlabel_X.shape[0])
    for i in range(train_unlabel_X.shape[0]):
        val[i]= w.dot(train_unlabel_X[i,:])+b
    y2=val.copy()
    y2[np.argsort(val)[::-1][:num_pos]]= 1
    y2[np.argsort(val)[::-1][num_pos:]]= -1
    
    # 4. Retrain with label switching
    C_neg = 1e-5
    C_pos = 1e-5 * num_pos / (X2.shape[0] - num_pos)
    while C_neg < C2 or C_pos < C2:
        
        # 5. Retrain the variant of SVM
        w, b, slack1, slack2= train_variantsvm(X1, y1, X2, y2, C1, C_neg, C_pos)
        
        # 6. Take a positive and negative example, switch their labels
        tmp_list=[]
        for i in range(len(y2)):
            for j in range(i+1, len(y2)):
                tmp_list.append((i,j))
        for l,m in tmp_list:
            while y2[l]*y2[l+1]<0 and slack2[l]>0 and slack2[l+1]>0 and slack2[l]+slack2[l+1]>2 :
                y2[l]= -y2[l]
                y2[m]= -y1[m]
                w, b, slack1, slack2= train_variantsvm(X1, y1, X2, y2, C1, C_neg, C_pos)
        # 7. Increase the value of C_neg and C_pos
        C_neg= min(2*C_neg, C1)
        C_pos= min(2*C_pos, C1)

    # 8. Return the learned model
    
    return w, b






In [3]:
test = load_svmlight_file('test.dat')
train = load_svmlight_file('train.dat')
test_X = np.array(test[0].todense())
test_y = test[1]
train_X = np.array(train[0].todense())
train_y = train[1]

print('train_X', train_X.shape)
print('train_y', train_y.shape)
print('test_X', test_X.shape)
print('test_y', test_y.shape)

train_X (610, 9930)
train_y (610,)
test_X (600, 9930)
test_y (600,)


In [4]:
# Extract labeled and unlabeled training samples
# 1, -1 indicate labeled samples
# 0 indicates unlabeled samples
train_label_X = train_X[np.where(train_y!=0)]
train_label_y = train_y[np.where(train_y!=0)]
train_unlabel_X = train_X[np.where(train_y==0)]

# Perform inductive linear SVM
print("Inductive SVM")
w, b = train_svm(train_label_X, train_label_y, C=1)
pred_y = np.sign(np.dot(test_X, w) + b)
print("Accuracy: %s\n" % accuracy_score(test_y, pred_y))

# Split the training set into two parts
n = train_label_X.shape[0] // 2
train_label_X1 = train_label_X[:n]
train_label_y1 = train_label_y[:n]
train_label_X2 = train_label_X[n:]
train_label_y2 = train_label_y[n:]

# Perform a variant of inductive linear SVM
print("Inductive SVM variate")
w, b, slack1, slack2= train_variantsvm(train_label_X1, train_label_y1, train_label_X2, train_label_y2, C1=1, C2=1, C3=1)
pred_y = np.sign(np.dot(test_X, w) + b)
print("Accuracy: %s\n" % accuracy_score(test_y, pred_y))

# Perform transductive linear SVM
print("Transductive SVM")
w, b = train_tsvm(train_label_X, train_label_y, train_unlabel_X, C1=1, C2=0.01, p=0.5)
pred_y = np.sign(np.dot(test_X, w) + b)
print("Accuracy: %s" % accuracy_score(test_y, pred_y))

Inductive SVM
Accuracy: 0.855

Inductive SVM variate
Accuracy: 0.855

Transductive SVM
Accuracy: 0.8933333333333333
